In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import time

from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.metrics import max_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor  
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

## Data loading

In [ ]:
df = pd.read_csv('data\Life-Expectancy-Data-Updated.csv')

# Simple renaming to improve readability
df=df.rename(columns={'Thinness_ten_nineteen_years':'Thinness (10-19 years)',\
                      'Thinness_five_nine_years':'Thinness (5-9 years)', \
                      'Economy_status_Developed' : 'Developed', \
                      'Economy_status_Developing' : 'Developing'                  
                     })
df_reduced = df.drop(['Infant_deaths', 'Under_five_deaths', 'Diphtheria', 'Thinness (5-9 years)', 'Developing'], axis=1)
df_reduced = df_reduced
df_reduced.head()

## Model selection

To use a reduced set of features set the following to True. To use all numerical features, except 'Year' set to false

In [ ]:
# Returns (X, y)
def split(df):
    reduced_features = False
    if(reduced_features):
        X = df[['Adult_mortality', 'Schooling', 'GDP_per_capita', 'Thinness (10-19 years)', 'Polio']]
    else:
        X = df.drop(columns=['Country', 'Region', 'Year', 'Life_expectancy'])
        
    y = df['Life_expectancy']
    return X, y

### PCA Analysis

In [ ]:
def pca_analysis(df):
    X, _ = split(df)
    X_scaled = StandardScaler().fit_transform(X)
    pca = PCA()
    _ = pca.fit_transform(X_scaled)
    
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Components')
    plt.ylabel('Fraction of Total Variance')

    pca_components = PCA().fit_transform(X_scaled)
    plt.figure(figsize=(8,6))
    sns.scatterplot(x=pca_components[:, 0], y=pca_components[:, 1], hue=df['Life_expectancy'], palette='viridis', alpha=0.75);
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')

    plt.show()

pca_analysis(df_reduced)

In [ ]:
df_train, df_test = train_test_split(df_reduced, test_size=0.2, random_state=42)

### Model comparison

In [ ]:
def compare_models(models):
    X_train, y_train = split(df_train)
    X_test, y_test = split(df_test)
    results = []
    for (name, model) in models:
        # Scale inputs - irrelevant for decision tree based models but important for KNN
        scaler = StandardScaler().fit(X_train)
        X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
        X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns) 
        
        t0 = time.time()
        count = 10
        for _ in range(count):
            model.fit(X_train_scaled, y_train)
            y_predict = model.predict(X_test_scaled)
        t1 = time.time()
        dict = {
                    'Name': name,
                    'Mean squared error': mean_squared_error(y_predict, y_test),
                    'Mean absolute error': mean_absolute_error(y_predict, y_test),
                    'Mean Absolute Percentage Error': mean_absolute_percentage_error(y_predict, y_test),
                    'Maximum Error': max_error(y_predict, y_test),
                    'r2 Score': r2_score(y_predict, y_test), 
                    'Time': (t1-t0)/count
                }
    
        results.append(dict)
    return pd.DataFrame(results).set_index('Name').sort_values('Mean squared error')

compare_models(
        { 
            ('Linear', LinearRegression()), 
            ('Decision Tree', DecisionTreeRegressor(random_state=42)),
            ('Random Forest', RandomForestRegressor(random_state=42)),
            ('Gradient Boosting', GradientBoostingRegressor(random_state=42)),
            ('XG Boost', XGBRegressor()),
            ('KNN', KNeighborsRegressor())
        })

Random Forest has slightly better MSE than XGBoost however the time for execution for XGBoost is very much faster. This is a significant benefit when performing scenario analysis using the model.

**We select the XGBoost model**

### Optimising XGBoost model

We use cross-validation to find the optimal parameters

### Cross validation

In [ ]:
from sklearn.base import clone
def score_folds(model, df, n_splits, random_state=42):
    scores = []
    kf = KFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    for i, (train_index, test_index) in enumerate(kf.split(df)):
        X_train, y_train = split(df.iloc[train_index]) # [columns]
        X_test, y_test = split(df.iloc[test_index]) # [columns]
        model_internal = clone(model)
        model_internal.fit(X_train, y_train)
        y_predict = model_internal.predict(X_test)
        scores.append(mean_squared_error(y_predict, y_test))
    return scores

In [ ]:
def scorer(params):
    model = XGBRegressor(**params)
    scores = score_folds(model, df_reduced, 5)
    mean = np.mean(scores)
    print(params, mean)
    return mean

param_grid={'learning_rate': [0.05, 0.1, 0.2],
            'max_depth':[6, 8, 10]}

grid = ParameterGrid(param_grid)
it2 = map(lambda p: (p, scorer(p)), grid)
best_params, _ = min(it2, key=lambda t: t[1])
print(best_params)

In [ ]:
model = XGBRegressor(**best_params).fit(*split(df_train))

## Model results

### Plots

In [ ]:
def plot_results(df):
    (X, y) = split(df)
    y_predict = model.predict(X)
    plt.scatter(y_predict, y)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [ ]:
plot_results(df_test)

In [ ]:
plot_results(df_test[df_test['Developed']==1])

In [ ]:
plot_results(df_test[df_test['Developed']==0])

In [ ]:
plot_results(df_test[df_test['Region']=='Africa'])

### Permutation importance

In [ ]:
def perm_importance(df):
    (X, y) = split(df)
    result = permutation_importance(model, X, y, n_repeats=100, random_state=42, n_jobs=2)
    idx = result.importances_mean.argsort()
    return pd.DataFrame(result.importances[idx].T, columns=X.columns[idx])

In [ ]:
importance_train = perm_importance(df_train)
importance_test = perm_importance(df_test)

In [ ]:
ax = importance_train.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (Training Set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()
plt.show()

In [ ]:
ax = importance_test.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (Test Set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()
plt.show()

In [ ]:
def plot_h(train, test):
    ax = plt.subplot(111)
    ind = np.arange(len(train.index))
    height=0.3
    ax.barh(ind+height/2, train, height, label='Train')
    ax.barh(ind-height/2, test, height, label='Test')
    ax.set(yticks=ind, yticklabels=train.index, ylim=[-2*height, len(ind)-height])
    ax.set_xlabel("Decrease in accuracy score")
    ax.figure.tight_layout()
    plt.legend(loc='lower right')
    plt.show()

plot_h(importance_train.mean(axis=0), importance_test.mean(axis=0))

In [ ]:
def plot_importance(df):
    importance_test = perm_importance(df)
    mean = importance_test.mean(axis=0)
    mean.plot.barh(stacked=True)
    plt.show()

In [ ]:
plot_importance(df_test[df_test['Developed']==1])

In [ ]:
plot_importance(df_test[df_test['Developed']==0])

In [ ]:
plot_importance(df_test[df_test['Region']=='Africa'])

### Partial dependence

In [ ]:
def partial_dep(df, feature):
    X, _ = split(df)
    PartialDependenceDisplay.from_estimator(model, X, [feature], kind='both')
    plt.show()

In [ ]:
partial_dep(df_test, 'Adult_mortality')

In [ ]:
partial_dep(df_test, 'GDP_per_capita')

In [ ]:
partial_dep(df_test[df_test['Developed']==0], 'Schooling')

In [ ]:
partial_dep(df_test[df_test['Developed']==0], 'Polio')